# Setup

This notebook is used to test scraping the model details, and updating the database for the `models` table.

In [1]:
import json
import os
import sys
from pathlib import Path

from bs4 import BeautifulSoup
import requests
import pandas as pd
import toml

from typing import List
from dataclasses import dataclass
# import attr

In [2]:
SRC = Path('').resolve().parents[2]  # ROOT folder -> ./src
LIB_PATH = SRC / "lib"

if str(LIB_PATH) not in sys.path:
    sys.path.insert(0, str(LIB_PATH))  # ./lib
else:
    pass

from path_desc import TFOD_MODELS_TABLE_PATH, CLASSIF_MODELS_NAME_PATH, SEGMENT_MODELS_TABLE_PATH

# Parse Secrets TOML file

In [3]:
toml_filepath = "../../../../.streamlit/secrets.toml"
parsed = toml.loads(open(toml_filepath).read())
parsed

{'postgres': {'host': 'localhost',
  'port': '5432',
  'dbname': 'integrated_vision_inspection_system',
  'user': 'postgres',
  'password': 'shrdc'}}

In [4]:
def parse_kwargs(**kwargs):
    for k, v in kwargs.items():
        print(k)
        print(v)

parse_kwargs(**parsed)

postgres
{'host': 'localhost', 'port': '5432', 'dbname': 'integrated_vision_inspection_system', 'user': 'postgres', 'password': 'shrdc'}


# Connect to PostgreSQL db

In [5]:
import psycopg2

conn = psycopg2.connect(**parsed['postgres'])

In [6]:
from psycopg2.extras import NamedTupleCursor

with conn.cursor(cursor_factory=NamedTupleCursor) as cur:
    cur.execute('SELECT version();')
    conn.commit()

    # display the PostgreSQL database server version
    db_version = cur.fetchone().version

print(f"PostgreSQL database version: {db_version}")
print(f"PostgreSQL connection status: {conn.info.status}")
print(
    f"You are connected to database '{conn.info.dbname}' as user '{conn.info.user}' on host '{conn.info.host}' at port '{conn.info.port}'.")

PostgreSQL database version: PostgreSQL 13.4, compiled by Visual C++ build 1914, 64-bit
PostgreSQL connection status: 0
You are connected to database 'integrated_vision_inspection_system' as user 'postgres' on host 'localhost' at port '5432'.


# DB Functions

In [19]:
# https://naysan.ca/2020/05/09/pandas-to-postgresql-using-psycopg2-bulk-insert-performance-benchmark/
# not used in this notebook
def copy_from_file(conn, df, table):
    """
    Here we are going save the dataframe on disk as
    a csv file, load the csv file
    and use copy_from() to copy it to the table
    """
    # Save the dataframe to disk
    tmp_df = "./tmp_dataframe.csv"
    df.to_csv(tmp_df)
    f = open(tmp_df, 'r')
    cursor = conn.cursor()
    try:
        cursor.copy_from(f, table, sep=",")
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        os.remove(tmp_df)
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    print("copy_from_file() done")
    cursor.close()
    os.remove(tmp_df)

In [27]:
from collections import namedtuple

def db_fetchone(sql_message, query_vars=None, conn=conn,
                return_output=False, fetch_col_name=False,
                return_dict=False) -> namedtuple:
    with conn.cursor(cursor_factory=NamedTupleCursor) as cur:
        try:
            if query_vars:
                cur.execute(sql_message, query_vars)
            else:
                cur.execute(sql_message)

            conn.commit()
            if not return_output:
                return
            return_one = cur.fetchone()  # return tuple
            # Obtain Column names from query
            column_names = [desc[0] for desc in cur.description]

            if return_dict:
                # Convert results to pure Python dictionary
                return_one = return_one._asdict()

            if fetch_col_name:
                return return_one, column_names
            else:
                return return_one
        except psycopg2.Error as e:
            conn.rollback()
            print(e)

In [28]:
sql_query = """
SELECT * FROM project
"""
output = db_fetchone(sql_query, return_output=True)
print(type(output))
print(output)

<class 'psycopg2.extras.Record'>
Record(id=1, name='test 1', description='test', deployment_id=1, created_at=datetime.datetime(2021, 9, 27, 14, 4, 19, 299862, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=480, name=None)), updated_at=datetime.datetime(2021, 9, 27, 14, 4, 19, 299862, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=480, name=None)))


In [15]:
output.deployment_id

1

In [31]:
%%time
output = db_fetchone(sql_query, return_output=True, return_dict=True)
print(type(output))
print(output)

<class 'dict'>
{'id': 1, 'name': 'test 1', 'description': 'test', 'deployment_id': 1, 'created_at': datetime.datetime(2021, 9, 27, 14, 4, 19, 299862, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=480, name=None)), 'updated_at': datetime.datetime(2021, 9, 27, 14, 4, 19, 299862, tzinfo=psycopg2.tz.FixedOffsetTimezone(offset=480, name=None))}
Wall time: 1 ms


## Delete existing pretrained_models record

In [32]:
# Delete existing pretrained models before inserting
sql_query = """
DELETE FROM models WHERE model_type_id = 1
"""
db_fetchone(sql_query)

In [23]:
sql_query = """
DROP SEQUENCE IF EXISTS models_sequence;

CREATE SEQUENCE models_sequence
  start 1
  increment 1;
"""
db_fetchone(sql_query)

# Scraping TFOD Model Zoo

In [24]:
link = "https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md"
df = pd.read_html(link)[0]
df

,Model name,Speed (ms),COCO mAP,Outputs
0,CenterNet HourGlass104 512x512,70,41.9,Boxes
1,CenterNet HourGlass104 Keypoints 512x512,76,40.0/61.4,Boxes/Keypoints
2,CenterNet HourGlass104 1024x1024,197,44.5,Boxes
3,CenterNet HourGlass104 Keypoints 1024x1024,211,42.8/64.5,Boxes/Keypoints
4,CenterNet Resnet50 V1 FPN 512x512,27,31.2,Boxes
5,CenterNet Resnet50 V1 FPN Keypoints 512x512,30,29.3/50.7,Boxes/Keypoints
6,CenterNet Resnet101 V1 FPN 512x512,34,34.2,Boxes
7,CenterNet Resnet50 V2 512x512,27,29.5,Boxes
8,CenterNet Resnet50 V2 Keypoints 512x512,30,27.6/48.2,Boxes/Keypoints
9,CenterNet MobileNetV2 FPN 512x512,6,23.4,Boxes


In [25]:
data = requests.get(link)
soup = BeautifulSoup(data.content, "html.parser")
print(soup.prettify())

<!DOCTYPE html>
<html data-color-mode="auto" data-dark-theme="dark" data-light-theme="light" lang="en">
 <head>
  <meta charset="utf-8"/>
  <link href="https://github.githubassets.com" rel="dns-prefetch"/>
  <link href="https://avatars.githubusercontent.com" rel="dns-prefetch"/>
  <link href="https://github-cloud.s3.amazonaws.com" rel="dns-prefetch"/>
  <link href="https://user-images.githubusercontent.com/" rel="dns-prefetch"/>
  <link crossorigin="" href="https://github.githubassets.com" rel="preconnect"/>
  <link href="https://avatars.githubusercontent.com" rel="preconnect"/>
  <link crossorigin="anonymous" href="https://github.githubassets.com/assets/light-83dd85a7ce7efef3332487e09e928562.css" integrity="sha512-g92Fp85+/vMzJIfgnpKFYqJ8VrYPHFHaqHRTgF8llOI+TnY0Ey75gxhQtNyXtv67Zv3ub5HufJFDJpvPPMO4UQ==" media="all" rel="stylesheet">
   <link crossorigin="anonymous" href="https://github.githubassets.com/assets/dark-296669110e982427b59cff62e3e550c0.css" integrity="sha512-KWZpEQ6YJCe1nP9i

In [26]:
model_links = soup.select("td a[href]")
print(len(model_links))
model_links[:5]

43


[<a href="http://download.tensorflow.org/models/object_detection/tf2/20200713/centernet_hg104_512x512_coco17_tpu-8.tar.gz" rel="nofollow">CenterNet HourGlass104 512x512</a>,
 <a href="http://download.tensorflow.org/models/object_detection/tf2/20200711/centernet_hg104_512x512_kpts_coco17_tpu-32.tar.gz" rel="nofollow">CenterNet HourGlass104 Keypoints 512x512</a>,
 <a href="http://download.tensorflow.org/models/object_detection/tf2/20200713/centernet_hg104_1024x1024_coco17_tpu-32.tar.gz" rel="nofollow">CenterNet HourGlass104 1024x1024</a>,
 <a href="http://download.tensorflow.org/models/object_detection/tf2/20200711/centernet_hg104_1024x1024_kpts_coco17_tpu-32.tar.gz" rel="nofollow">CenterNet HourGlass104 Keypoints 1024x1024</a>,
 <a href="http://download.tensorflow.org/models/object_detection/tf2/20200711/centernet_resnet50_v1_fpn_512x512_coco17_tpu-8.tar.gz" rel="nofollow">CenterNet Resnet50 V1 FPN 512x512</a>]

In [27]:
model_links = [link['href'] for link in model_links]
model_links[:5]

['http://download.tensorflow.org/models/object_detection/tf2/20200713/centernet_hg104_512x512_coco17_tpu-8.tar.gz',
 'http://download.tensorflow.org/models/object_detection/tf2/20200711/centernet_hg104_512x512_kpts_coco17_tpu-32.tar.gz',
 'http://download.tensorflow.org/models/object_detection/tf2/20200713/centernet_hg104_1024x1024_coco17_tpu-32.tar.gz',
 'http://download.tensorflow.org/models/object_detection/tf2/20200711/centernet_hg104_1024x1024_kpts_coco17_tpu-32.tar.gz',
 'http://download.tensorflow.org/models/object_detection/tf2/20200711/centernet_resnet50_v1_fpn_512x512_coco17_tpu-8.tar.gz']

In [28]:
df['model_links'] = model_links
df

,Model name,Speed (ms),COCO mAP,Outputs,model_links
0,CenterNet HourGlass104 512x512,70,41.9,Boxes,http://download.tensorflow.org/models/object_d...
1,CenterNet HourGlass104 Keypoints 512x512,76,40.0/61.4,Boxes/Keypoints,http://download.tensorflow.org/models/object_d...
2,CenterNet HourGlass104 1024x1024,197,44.5,Boxes,http://download.tensorflow.org/models/object_d...
3,CenterNet HourGlass104 Keypoints 1024x1024,211,42.8/64.5,Boxes/Keypoints,http://download.tensorflow.org/models/object_d...
4,CenterNet Resnet50 V1 FPN 512x512,27,31.2,Boxes,http://download.tensorflow.org/models/object_d...
5,CenterNet Resnet50 V1 FPN Keypoints 512x512,30,29.3/50.7,Boxes/Keypoints,http://download.tensorflow.org/models/object_d...
6,CenterNet Resnet101 V1 FPN 512x512,34,34.2,Boxes,http://download.tensorflow.org/models/object_d...
7,CenterNet Resnet50 V2 512x512,27,29.5,Boxes,http://download.tensorflow.org/models/object_d...
8,CenterNet Resnet50 V2 Keypoints 512x512,30,27.6/48.2,Boxes/Keypoints,http://download.tensorflow.org/models/object_d...
9,CenterNet MobileNetV2 FPN 512x512,6,23.4,Boxes,http://download.tensorflow.org/models/object_d...


In [29]:
# df.to_parquet('tfod_models.parquet', index=False)
df.columns = ["Model Name", "Speed (ms)", "COCO mAP",
                  "Outputs", "model_links"]
df.to_csv(TFOD_MODELS_TABLE_PATH, index=False)

In [30]:
# df = pd.read_parquet('tfod_models.parquet')
df = pd.read_csv(TFOD_MODELS_TABLE_PATH)
df.head()

,Model name,Speed (ms),COCO mAP,Outputs,model_links
0,CenterNet HourGlass104 512x512,70,41.9,Boxes,http://download.tensorflow.org/models/object_d...
1,CenterNet HourGlass104 Keypoints 512x512,76,40.0/61.4,Boxes/Keypoints,http://download.tensorflow.org/models/object_d...
2,CenterNet HourGlass104 1024x1024,197,44.5,Boxes,http://download.tensorflow.org/models/object_d...
3,CenterNet HourGlass104 Keypoints 1024x1024,211,42.8/64.5,Boxes/Keypoints,http://download.tensorflow.org/models/object_d...
4,CenterNet Resnet50 V1 FPN 512x512,27,31.2,Boxes,http://download.tensorflow.org/models/object_d...


Need columns: id, name, description, metrics, model_path, model_type_id, framework_id, deployment_id

In [31]:
new_df = df[['Model name']].copy()
new_df['metrics'] = json.dumps({'metrics': 'mAP'})
new_df['model_type_id'] = 1  # for pretrained_model type
new_df['framework_id'] = 1  # for Tensorflow type
new_df['deployment_id'] = 2  # for Object Detection
new_df.rename(columns={'Model name': 'name'}, inplace=True)
new_df

,name,metrics,model_type_id,framework_id,deployment_id
0,CenterNet HourGlass104 512x512,"{""metrics"": ""mAP""}",1,1,2
1,CenterNet HourGlass104 Keypoints 512x512,"{""metrics"": ""mAP""}",1,1,2
2,CenterNet HourGlass104 1024x1024,"{""metrics"": ""mAP""}",1,1,2
3,CenterNet HourGlass104 Keypoints 1024x1024,"{""metrics"": ""mAP""}",1,1,2
4,CenterNet Resnet50 V1 FPN 512x512,"{""metrics"": ""mAP""}",1,1,2
5,CenterNet Resnet50 V1 FPN Keypoints 512x512,"{""metrics"": ""mAP""}",1,1,2
6,CenterNet Resnet101 V1 FPN 512x512,"{""metrics"": ""mAP""}",1,1,2
7,CenterNet Resnet50 V2 512x512,"{""metrics"": ""mAP""}",1,1,2
8,CenterNet Resnet50 V2 Keypoints 512x512,"{""metrics"": ""mAP""}",1,1,2
9,CenterNet MobileNetV2 FPN 512x512,"{""metrics"": ""mAP""}",1,1,2


## Insert to DB

In [32]:
def insert_to_db(new_df):
    sql_query = """
                INSERT INTO public.models (
                    id,
                    name,
                    metrics,
                    model_type_id,
                    framework_id,
                    deployment_id
                    )
                VALUES (
                    nextval('models_sequence'),
                    %s,
                    %s,
                    %s,
                    %s,
                    %s
                    )
    """
    for row in new_df.values:
        query_vars = row.tolist()
        db_fetchone(sql_query, query_vars)

In [33]:
insert_to_db(new_df)

# Scraping Keras pretrained model functions

In [34]:
URL = "https://www.tensorflow.org/api_docs/python/tf/keras/applications"
data = requests.get(URL)
soup = BeautifulSoup(data.text, 'html.parser')

In [35]:
names = soup.select("code")
model_names = []
for i in names:
    if '(...)' in i.text:
        link = i.text.replace('(...)', '')
        model_names.append(link)
model_names

['DenseNet121',
 'DenseNet169',
 'DenseNet201',
 'EfficientNetB0',
 'EfficientNetB1',
 'EfficientNetB2',
 'EfficientNetB3',
 'EfficientNetB4',
 'EfficientNetB5',
 'EfficientNetB6',
 'EfficientNetB7',
 'InceptionResNetV2',
 'InceptionV3',
 'MobileNet',
 'MobileNetV2',
 'MobileNetV3Large',
 'MobileNetV3Small',
 'NASNetLarge',
 'NASNetMobile',
 'ResNet101',
 'ResNet101V2',
 'ResNet152',
 'ResNet152V2',
 'ResNet50',
 'ResNet50V2',
 'VGG16',
 'VGG19',
 'Xception']

In [36]:
df = pd.DataFrame(model_names, columns=['name'])
df

,name
0,DenseNet121
1,DenseNet169
2,DenseNet201
3,EfficientNetB0
4,EfficientNetB1
5,EfficientNetB2
6,EfficientNetB3
7,EfficientNetB4
8,EfficientNetB5
9,EfficientNetB6


In [37]:
df['metrics'] = json.dumps({"metrics": ["Accuracy", "F1 Score"]})  # for pretrained_model type
df['model_type_id'] = 1  # for pretrained_model type
df['framework_id'] = 1  # for Tensorflow type
df['deployment_id'] = 1  # for image classification
df

,name,metrics,model_type_id,framework_id,deployment_id
0,DenseNet121,"{""metrics"": [""Accuracy"", ""F1 Score""]}",1,1,1
1,DenseNet169,"{""metrics"": [""Accuracy"", ""F1 Score""]}",1,1,1
2,DenseNet201,"{""metrics"": [""Accuracy"", ""F1 Score""]}",1,1,1
3,EfficientNetB0,"{""metrics"": [""Accuracy"", ""F1 Score""]}",1,1,1
4,EfficientNetB1,"{""metrics"": [""Accuracy"", ""F1 Score""]}",1,1,1
5,EfficientNetB2,"{""metrics"": [""Accuracy"", ""F1 Score""]}",1,1,1
6,EfficientNetB3,"{""metrics"": [""Accuracy"", ""F1 Score""]}",1,1,1
7,EfficientNetB4,"{""metrics"": [""Accuracy"", ""F1 Score""]}",1,1,1
8,EfficientNetB5,"{""metrics"": [""Accuracy"", ""F1 Score""]}",1,1,1
9,EfficientNetB6,"{""metrics"": [""Accuracy"", ""F1 Score""]}",1,1,1


## Insert to DB

In [38]:
insert_to_db(df)

# (Removed) Scraping segmentation_models library's Backbones

In [39]:
URL = 'https://github.com/qubvel/segmentation_models'
df_list = pd.read_html(URL)
df_list

[   Unet  Linknet
 0   NaN      NaN,
    PSPNet  FPN
 0     NaN  NaN,
            Type                                              Names
 0           VGG                                    'vgg16' 'vgg19'
 1        ResNet  'resnet18' 'resnet34' 'resnet50' 'resnet101' '...
 2     SE-ResNet  'seresnet18' 'seresnet34' 'seresnet50' 'seresn...
 3       ResNeXt                           'resnext50' 'resnext101'
 4    SE-ResNeXt                       'seresnext50' 'seresnext101'
 5      SENet154                                         'senet154'
 6      DenseNet          'densenet121' 'densenet169' 'densenet201'
 7     Inception                  'inceptionv3' 'inceptionresnetv2'
 8     MobileNet                          'mobilenet' 'mobilenetv2'
 9  EfficientNet  'efficientnetb0' 'efficientnetb1' 'efficientne...]

In [40]:
for table in df_list:
    if table.columns.tolist() == ['Type', 'Names']:
        df = table.copy()
df

,Type,Names
0,VGG,'vgg16' 'vgg19'
1,ResNet,'resnet18' 'resnet34' 'resnet50' 'resnet101' '...
2,SE-ResNet,'seresnet18' 'seresnet34' 'seresnet50' 'seresn...
3,ResNeXt,'resnext50' 'resnext101'
4,SE-ResNeXt,'seresnext50' 'seresnext101'
5,SENet154,'senet154'
6,DenseNet,'densenet121' 'densenet169' 'densenet201'
7,Inception,'inceptionv3' 'inceptionresnetv2'
8,MobileNet,'mobilenet' 'mobilenetv2'
9,EfficientNet,'efficientnetb0' 'efficientnetb1' 'efficientne...


In [41]:
df['Names'] = df['Names'].str.replace(' ', ', ')
df.rename(columns={'Names': 'Variants'}, inplace=True)
df

,Type,Variants
0,VGG,"'vgg16', 'vgg19'"
1,ResNet,"'resnet18', 'resnet34', 'resnet50', 'resnet101..."
2,SE-ResNet,"'seresnet18', 'seresnet34', 'seresnet50', 'ser..."
3,ResNeXt,"'resnext50', 'resnext101'"
4,SE-ResNeXt,"'seresnext50', 'seresnext101'"
5,SENet154,'senet154'
6,DenseNet,"'densenet121', 'densenet169', 'densenet201'"
7,Inception,"'inceptionv3', 'inceptionresnetv2'"
8,MobileNet,"'mobilenet', 'mobilenetv2'"
9,EfficientNet,"'efficientnetb0', 'efficientnetb1', 'efficient..."


# Scraping keras-unet-collections models

In [42]:
URL = "https://github.com/yingkaisha/keras-unet-collection"
data = requests.get(URL)
soup = BeautifulSoup(data.text, 'html.parser')

In [43]:
tables = soup.select("table")
len(tables)

4

In [44]:
for table in tables:
    if 'models' in table.th.text:
        models_table = table
    if 'losses' in table.th.text:
        losses_table = table

In [45]:
model_links = models_table.select("td a")
model_links = [i['href'] for i in model_links]

In [46]:
model_df = pd.read_html(models_table.prettify())[0]

In [47]:
model_df['links'] = model_links
model_df.head()

,keras_unet_collection.models,Name,Reference,links
0,unet_2d,U-net,Ronneberger et al. (2015),https://link.springer.com/chapter/10.1007/978-...
1,vnet_2d,V-net (modified for 2-d inputs),Milletari et al. (2016),https://arxiv.org/abs/1606.04797
2,unet_plus_2d,U-net++,Zhou et al. (2018),https://link.springer.com/chapter/10.1007/978-...
3,r2_unet_2d,R2U-Net,Alom et al. (2018),https://arxiv.org/abs/1802.06955
4,att_unet_2d,Attention U-net,Oktay et al. (2018),https://arxiv.org/abs/1804.03999


In [48]:
model_df.rename(columns={"keras_unet_collection.models": "model_func"}, inplace=True)

In [49]:
# dropping the last two models built with Transformers as they don't work for new NumPy version,
# refer to the repo for more details
model_df.drop(model_df.index[-2:], inplace=True)
model_df

,model_func,Name,Reference,links
0,unet_2d,U-net,Ronneberger et al. (2015),https://link.springer.com/chapter/10.1007/978-...
1,vnet_2d,V-net (modified for 2-d inputs),Milletari et al. (2016),https://arxiv.org/abs/1606.04797
2,unet_plus_2d,U-net++,Zhou et al. (2018),https://link.springer.com/chapter/10.1007/978-...
3,r2_unet_2d,R2U-Net,Alom et al. (2018),https://arxiv.org/abs/1802.06955
4,att_unet_2d,Attention U-net,Oktay et al. (2018),https://arxiv.org/abs/1804.03999
5,resunet_a_2d,ResUnet-a,Diakogiannis et al. (2020),https://doi.org/10.1016/j.isprsjprs.2020.01.013
6,u2net_2d,U^2-Net,Qin et al. (2020),https://arxiv.org/abs/2005.09007
7,unet_3plus_2d,UNET 3+,Huang et al. (2020),https://arxiv.org/abs/2004.08790


In [50]:
model_df.columns = ["model_func", "Model Name", "Reference", "links"]
model_df.to_csv(SEGMENT_MODELS_TABLE_PATH, index=False)

In [51]:
loss_df = pd.read_html(losses_table.prettify())[0]
loss_df

,keras_unet_collection.losses,Name,Reference
0,dice,Dice loss,Sudre et al. (2017)
1,tversky,Tversky loss,Hashemi et al. (2018)
2,focal_tversky,Focal Tversky loss,Abraham et al. (2019)
3,ms_ssim,Multi-scale Structural Similarity Index loss,Wang et al. (2003)
4,iou_seg,Intersection over Union (IoU) loss for segment...,Rahman and Wang (2016)
5,iou_box,(Generalized) IoU loss for object detection,Rezatofighi et al. (2019)
6,triplet_1d,Semi-hard triplet loss (experimental),NaN
7,crps2d_tf,CRPS loss (experimental),NaN


In [52]:
experimental_row_idxs = loss_df.loc[loss_df['Name'].str.contains('experimental')].index
loss_df = loss_df.drop(experimental_row_idxs).reset_index(drop=True)

In [53]:
loss_links = losses_table.select("td a")
loss_links = [i['href'] for i in loss_links]

loss_df['links'] = loss_links
loss_df.rename(columns={"keras_unet_collection.losses": "loss_func"}, inplace=True)
loss_df

,loss_func,Name,Reference,links
0,dice,Dice loss,Sudre et al. (2017),https://link.springer.com/chapter/10.1007/978-...
1,tversky,Tversky loss,Hashemi et al. (2018),https://ieeexplore.ieee.org/abstract/document/...
2,focal_tversky,Focal Tversky loss,Abraham et al. (2019),https://ieeexplore.ieee.org/abstract/document/...
3,ms_ssim,Multi-scale Structural Similarity Index loss,Wang et al. (2003),https://ieeexplore.ieee.org/abstract/document/...
4,iou_seg,Intersection over Union (IoU) loss for segment...,Rahman and Wang (2016),https://link.springer.com/chapter/10.1007/978-...
5,iou_box,(Generalized) IoU loss for object detection,Rezatofighi et al. (2019),https://openaccess.thecvf.com/content_CVPR_201...


In [54]:
df = model_df[['Name']].copy()
df['metrics'] = json.dumps({"metrics": "mAP", "losses": loss_df.loss_func.tolist()})
df['model_type_id'] = 1  # for pretrained_model type
df['framework_id'] = 1  # for Tensorflow type
df['deployment_id'] = 3  # for image segmentation
df

,Name,metrics,model_type_id,framework_id,deployment_id
0,U-net,"{""metrics"": ""mAP"", ""losses"": [""dice"", ""tversky...",1,1,3
1,V-net (modified for 2-d inputs),"{""metrics"": ""mAP"", ""losses"": [""dice"", ""tversky...",1,1,3
2,U-net++,"{""metrics"": ""mAP"", ""losses"": [""dice"", ""tversky...",1,1,3
3,R2U-Net,"{""metrics"": ""mAP"", ""losses"": [""dice"", ""tversky...",1,1,3
4,Attention U-net,"{""metrics"": ""mAP"", ""losses"": [""dice"", ""tversky...",1,1,3
5,ResUnet-a,"{""metrics"": ""mAP"", ""losses"": [""dice"", ""tversky...",1,1,3
6,U^2-Net,"{""metrics"": ""mAP"", ""losses"": [""dice"", ""tversky...",1,1,3
7,UNET 3+,"{""metrics"": ""mAP"", ""losses"": [""dice"", ""tversky...",1,1,3


## Insert to DB

In [55]:
insert_to_db(df)